In [2]:
import cv2
import os
import numpy as np
import random

input_file = 'VDB\D.mp4'   

# Open Video

In [108]:
def open_vid(input_file):
    cap = cv2.VideoCapture(input_file)
    if not cap.isOpened():
        print("Error: Could not open video.")
    return cap

In [ ]:
cap = open_vid(input_file)

# Get Video Properties

In [4]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Width: ",width)
print("Height: ",height)
print("Original FPS: ",original_fps)
print("Frame Count: ",frame_count)

Width:  640
Height:  358
Original FPS:  30.0
Frame Count:  2834


In [5]:
def get_frames(cap):
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(frame_count):
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            return frames

        # Save the frame to the list
        frames.append(frame)
        
    if not frames:
        print("No frames were saved.")
    
    return frames

In [6]:
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
Frames = get_frames(cap)

# Delete PNG Files in Directory

In [7]:
def delete_png_files(directory_path):
    # List all files in the specified directory
    for filename in os.listdir(directory_path):
        # Construct full file path
        file_path = os.path.join(directory_path, filename)
        try:
            # Check if it's a PNG file and remove it
            if os.path.isfile(file_path) and filename.lower().endswith('.png'):
                os.remove(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

# Read and Save Frames in Interval

In [8]:
def save_frames_as_vid(frames, frame_folder):
    # Create the folder to save frames if it doesn't exist
    if not os.path.exists(frame_folder):
        os.makedirs(frame_folder)

    frame_count = 0
    for i in range(len(frames)):
        # Save the frame as an image file
        frame_filename = os.path.join(frame_folder, f'frame_{frame_count:03d}.png')
        cv2.imwrite(frame_filename, frames[i])
        frame_count += 1

    #if not frames:
         #print("No frames were saved.")

In [9]:
#delete_png_files('saved_frames')

In [10]:
#save_frames_as_vid(Frames[50:100],"saved_frames")

# Create Video with Set of Frames

In [11]:
def save_vid(frames,output_file,fps):
    # Get frame dimensions
    height, width, _ = frames[0].shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    # Write the frames to the new video file
    [out.write(frame) for frame in frames];

    # Release the video writer object
    out.release()

In [12]:
#save_vid(Frames[100:150], "slice_vid.mp4", original_fps)

# Change FPS

In [13]:
#save_vid(Frames[100:150],"slice_vid_new_fps.mp4",12)

# Play Frames

In [14]:
def play_frames(frames,fps):
    
    delay = int(1000/fps)
    print("Delay: ",delay)

    for frame in frames:
        # Display the frame
        cv2.imshow('Video Playback', frame)
        # Exit the playback if 'q' is pressed
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
            
    cv2.destroyAllWindows()

In [15]:
#play_frames(Frames[:100],24)

# Optical Flow

## Farneback Magnitude

In [16]:
def OFM(frames):
    
    OF = [] 
    #Convert to Grayscale
    prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)

    for i in range(len(frames)-1):
        #Convert the next frame to grayscale
        next_gray = cv2.cvtColor(frames[i+1], cv2.COLOR_BGR2GRAY)

        # Calculate the dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5,  3,  10,  10,    5,    1.2,   0)
                                            #prev      next       flow  dist lvl  win  it  smooth  std   flag 
 
        # Visualize the optical flow
        hsv = np.zeros_like(frames[i]) #[ch0,ch1,ch2]
        hsv[..., 1] = 255 #ch1 Saturation (Full)

        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1]) #Cartesian to Polar
        hsv[..., 0] = ang * 180 / np.pi / 2  
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX) #Normalize from 0 to 255
        flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)   #Convert to RBG
        OF.append(flow_rgb)
        
        flow_M  = flow[...,0]+flow[...,1]
        
        # Display the original frame and the optical flow
        cv2.imshow('Prev Frame', frames[i])
        cv2.imshow('Next Frame', frames[i+1])
        cv2.imshow('Optical Flow', flow_rgb)
        cv2.imshow('Optical Flow Mag',flow_M)

        # Wait for a key press to move to the next frame
        if cv2.waitKey(0) & 0xFF==ord('q'):
            break

        # Update the previous frame and grayscale image
        prev_gray = next_gray

    # Release the video capture object and close all OpenCV windows
    cv2.destroyAllWindows()
    return OF

In [17]:
Optical_R = OFM(Frames[100:150])

## Farneack with Vectors

In [18]:
def draw_optical_flow_vectors(flow, frame, step):
    h, w = frame.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # Create a mask to draw the vectors
    mask = np.zeros_like(frame)
    
    # Create line endpoints
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    # Draw lines and circles
    for (x1, y1), (x2, y2) in lines:
        cv2.line(mask, (x1, y1), (x2, y2), (0, 255, 0), 1)
        cv2.circle(frame, (x1, y1), 1, (0, 255, 0), -1)

    return cv2.add(frame, mask)

def OFV(frames,step):
    OF = []
    
    #convert it to grayscale
    prev_gray = cv2.cvtColor(np.copy(frames[0]), cv2.COLOR_BGR2GRAY)

    for i in range(1,len(frames)):
        # next frame convert it to grayscale
        next_gray = cv2.cvtColor(np.copy(frames[i]), cv2.COLOR_BGR2GRAY)

        # Calculate the dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Draw the optical flow vectors on the frame
        flow_frame = draw_optical_flow_vectors(flow, np.copy(frames[i]),step)
        OF.append(flow)
        
        # Display the original frame with optical flow vectors
        cv2.imshow('Prev Frame',frames[i])
        cv2.imshow('Next Frame',frames[i-1])
        cv2.imshow('Optical Flow Vectors', flow_frame)

        # Wait for a key press to move to the next frame
        if cv2.waitKey(0) & 0xFF == ord('q'):
            break

        # Update the previous frame and grayscale image
        prev_gray = next_gray

    # Release the video capture object and close all OpenCV windows
    cv2.destroyAllWindows()
    return OF

In [19]:
Optical_V = OFV(Frames[100:150],20)

## Lucas-Kanade

In [20]:
def OFKL(frames):
    # Parameters for the Lucas-Kanade optical flow
    lk_params = dict(winSize=(10, 10), maxLevel=3, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Take the first frame and convert it to grayscale
    old_frame = np.copy(frames[0])
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    # Create a grid of points to track
    step_size = 10
    grid_y, grid_x = np.mgrid[0:old_gray.shape[0]:step_size, 0:old_gray.shape[1]:step_size]
    p0 = np.vstack((grid_x.ravel(), grid_y.ravel())).T.astype(np.float32).reshape(-1, 1, 2)


    for i in range(1, len(frames)):
        frame = np.copy(frames[i])
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Select good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Draw the normalized vectors
        for j, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            # Calculate vector components
            dx = a - c
            dy = b - d
            # Normalize vector
            magnitude = np.sqrt(dx**2 + dy**2)
            if magnitude > 0:
                dx /= magnitude
                dy /= magnitude
            # Scale vector for visualization
            scale = 10  # Adjust this value for smaller or larger arrows
            a = int(c + dx * scale)
            b = int(d + dy * scale)
            frame = cv2.arrowedLine(frame, (int(c), int(d)), (a, b), (0, 255, 0), 2, tipLength=0.5)
        
        cv2.imshow('Prev',frames[i-1])
        cv2.imshow('Next',frames[i])
        cv2.imshow('Optical Flow Vectors', frame)
        
        if cv2.waitKey(0) & 0xff == ord('q'):
            break

        # Update the previous frame and previous points
        old_gray = frame_gray.copy()

    cv2.destroyAllWindows()

In [21]:
OFKL(Frames[100:150])

## Phase Correlation

In [22]:
def PhaseC(frames):

    for i in range(len(frames) - 1):
        prev_frame = frames[i]
        next_frame = frames[i + 1]

        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

        # Compute phase correlation
        shift, _ = cv2.phaseCorrelate(prev_gray.astype(np.float32), next_gray.astype(np.float32))
        
        # Visualize the shift as a vector field
        flow = np.zeros_like(prev_gray)
        h, w = flow.shape
        x, y = int(w/2), int(h/2)
        dx, dy = shift

        # Draw the vector on the flow image
        flow_img = cv2.cvtColor(flow, cv2.COLOR_GRAY2BGR)
        cv2.arrowedLine(flow_img, (x, y), (int(x + dx * 10), int(y + dy * 10)), (0, 255, 0), 2, tipLength=0.3)

        # Display the frames and the flow
        cv2.imshow('Previous Frame', prev_frame)
        cv2.imshow('Next Frame', next_frame)
        cv2.imshow('Optical Flow', flow_img)
        
        
        # Wait for key press to proceed to the next frame
        if cv2.waitKey(0) & 0xff == ord('q'):
            break

    cv2.destroyAllWindows()

In [23]:
PhaseC(Frames[100:150])

# Frames Differences

In [24]:
def frame_dif(frames, threshold=30):
    D = []
    prev_frame = frames[0]

    for i in range(1,len(frames)):

        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        # Compute absolute difference between frames
        diff = cv2.absdiff(prev_gray, curr_gray)
              
        # Create a mask to highlight pixels with significant changes
        mask = np.zeros_like(diff)
        mask[diff > threshold] = 255
        D.append(mask)
        # Show original frame and mask
        cv2.imshow('Original Frame', frames[i])
        cv2.imshow('Pixels with Most Changes', mask)
        if cv2.waitKey(0) & 0xff == ord('q'):
            break

        prev_frame = frames[i]

    # Release the video capture object
    cv2.destroyAllWindows()
    return D

In [25]:
Dif = frame_dif(Frames[200:250],20)

# Change Color Channel's Ranges

In [30]:
def change_range_colors(image, min_vals=(0, 0, 0), max_vals=(255, 255, 255)):
    # Split the image into its BGR channels
    b, g, r = cv2.split(image)
    
    # Clip each channel to its respective range
    b = np.clip(b, min_vals[0], max_vals[0])
    g = np.clip(g, min_vals[1], max_vals[1])
    r = np.clip(r, min_vals[2], max_vals[2])
    
    # Merge the channels back together
    new_image = cv2.merge((b, g, r))
    
    return new_image

In [43]:
Nframe = change_range_colors(Frames[100],(0,50,100),(250,200,250))
cv2.imshow('original',Frames[100])
cv2.imshow('changed',Nframe)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Add Occlusions

In [158]:
def occlusions(image, num_occlusions):    
    output_image = np.copy(image)
    
    height, width = image.shape[:2]
    
    # Draw random occlusions on the image
    for _ in range(num_occlusions):
        shape_type = random.choice(['rectangle', 'circle'])
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))  # Random color
        
        if shape_type == 'rectangle':
            x = random.randint(0, width - 1)
            y = random.randint(0, height - 1)
            width_rect = random.randint(10, 100)
            height_rect = random.randint(10, 100)
            cv2.rectangle(output_image, (x, y), (x + width_rect, y + height_rect), color, -1)  # Filled rectangle
        
        elif shape_type == 'circle':
            center = (random.randint(0, width - 1), random.randint(0, height - 1))
            radius = random.randint(5, 50)
            cv2.circle(output_image, center, radius, color, -1)  # Filled circle
    
    return output_image

In [159]:
New_img = draw_occlusions(Frames[100], 1)

cv2.imshow('Original Image', Frames[100])
cv2.imshow('Image with Occlusions', cv2.cvtColor(New_img,cv2.COLOR_BGR2RGB))
cv2.imshow('Image with Occlusions1', New_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Display Image

In [116]:
def display_frame(I):
    cv2.imshow('I', I)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Experiments

In [181]:
file = 'VDB\H.mp4'
cap = open_vid(file)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Width: ",width)
print("Height: ",height)
print("Original FPS: ",original_fps)
print("Frame Count: ",frame_count)
Frames = get_frames(cap)

Width:  640
Height:  346
Original FPS:  25.0
Frame Count:  3946


In [182]:
F = Frames.copy()
F[115] = cv2.cvtColor(draw_occlusions(F[115],1),cv2.COLOR_BGR2RGB)
display_frame(F[115])

In [184]:
play_frames(F[100:130],original_fps)

Delay:  40


In [188]:
play_frames(F[100:130],10)

Delay:  200


In [187]:
play_frames(F[100:130],50)

Delay:  20


In [190]:
F = Frames.copy()
F[115] = change_range_colors(F[115],(0,50,100),(250,200,250))
display_frame(F[115])

In [191]:
play_frames(F[100:130],original_fps)

Delay:  40


In [192]:
play_frames(F[100:130],10)

Delay:  100


In [193]:
play_frames(F[100:130],50)

Delay:  20
